In [ ]:
from google.colab import drive
# mount drive to access data
drive.mount('/content/drive')
# load data folder into working directory
!cp -r drive/MyDrive/data .

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# only execute when in colab!
# download pretrained word vectors and unzip
!mkdir embeddings
!wget -P embeddings https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M-subword.vec.zip
!wget -P embeddings https://huggingface.co/stanfordnlp/glove/resolve/main/glove.twitter.27B.zip

mkdir: cannot create directory ‘embeddings’: File exists
--2023-05-06 18:09:46--  https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M-subword.vec.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 108.157.162.83, 108.157.162.108, 108.157.162.35, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|108.157.162.83|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 587879973 (561M) [application/zip]
Saving to: ‘embeddings/wiki-news-300d-1M-subword.vec.zip’

wiki-news-300d-1M-s 100%[===================>] 560.65M   138MB/s    in 4.2s    

2023-05-06 18:09:50 (135 MB/s) - ‘embeddings/wiki-news-300d-1M-subword.vec.zip’ saved [587879973/587879973]

--2023-05-06 18:09:51--  https://huggingface.co/stanfordnlp/glove/resolve/main/glove.twitter.27B.zip
Resolving huggingface.co (huggingface.co)... 18.67.0.67, 18.67.0.34, 18.67.0.55, ...
Connecting to huggingface.co (huggingface.co)|18.67.0.67|:443... connected.
HTTP request sent

In [ ]:
!unzip -d embeddings embeddings/wiki-news-300d-1M-subword.vec.zip

Archive:  embeddings/wiki-news-300d-1M-subword.vec.zip
  inflating: embeddings/wiki-news-300d-1M-subword.vec  


In [ ]:
!unzip -d embeddings embeddings/glove.twitter.27B.zip

Archive:  embeddings/glove.twitter.27B.zip
replace embeddings/glove.twitter.27B.100d.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import nltk
from nltk.tokenize import TweetTokenizer
from nltk.stem import WordNetLemmatizer
import pandas as pd
import matplotlib.pyplot as plt
import string
import re
from nltk.util import bigrams
from itertools import compress
import numpy as np
import math
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm
import io

In [ ]:
# loading data with preprocessed tweets
data = pd.read_csv('data/train.csv')

In [ ]:
tweets = []
for ind, row in tqdm(data.iterrows(), total=data.shape[0]):
  try:
    tweets.append(row['TweetText'].split())
  except:
    print(row['TweetText'])

100%|██████████| 449366/449366 [00:41<00:00, 10954.17it/s]


### Bag of Words

In [ ]:
vectorizer = CountVectorizer(max_features=1000)
X = vectorizer.fit_transform(data['TweetText']).toarray()

In [ ]:
vectorizer.vocabulary_

### TF-IDF

In [ ]:
vectorizer = TfidfVectorizer(max_features=1000)
X = vectorizer.fit_transform(data['TweetText'])

In [ ]:
X.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

### Word2Vec

### FastText

In [ ]:
def load_vectors(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    data = {}
    for line in fin:
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = np.asarray(tokens[1:], dtype=np.float32)
    return data

In [ ]:
ft_embeddings = load_vectors("embeddings/wiki-news-300d-1M-subword.vec")

[ 0.0203 -0.0123 -0.0076  0.0187  0.0173 -0.0052  0.0147  0.1364 -0.0435
  0.0033]
[ 0.0203 -0.0123 -0.0076  0.0187  0.0173 -0.0052  0.0147  0.1364 -0.0435
  0.0033]


### GloVe

In [ ]:
glove_embeddings = {}
with open("embeddings/glove.twitter.27B.200d.txt", 'r') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], "float32")
        glove_embeddings[word] = vector

In [ ]:
glove_embeddings